In [1]:
from helloga.environment import Environment
from helloga.individual import BinaryIndividual 
from helloga.crossover import SinglePointCrossOver
from helloga.selector import LeadingSelector
from helloga.fitness import WeightedSumFitness
import logging 
import numpy as np

# Formulate the Problem
Assume we have 12 goods to be packed into a travel suitcase. The suitcase can hold at list 250 kg goods. We have measured the weights for each of the goods and define importance so that we should packed more important goods as much as possible in the limit of weight constraint.   

Define the chromosome as a 0-1 vector to represent each box. The problem is to find out a vector that maximize sum of box ***I***mportance where sum of box ***W***eights are less than or equal to 250. eq.   
$$ \max \sum_{i}{I_i} \quad where \quad i \in \{0, \dots, 11\}  $$
$$ s.t. \sum_{i}{W_i} <= 250 $$

# Define the algorithm
## Intial parameters 
1. Chromosome: binary vector to represent whether the box is selected to put into my bag. eg. [1,1,0,0,0,0,0,0,0,1,1,0]
2. Importance: a vector with same length and constant values: [6, 5, 8, 7, 6, 9, 4, 5, 4, 9, 2, 1]
3. Box Weight: a vector with same length and constant values: [20, 30, 60, 90, 50, 70, 30, 30, 70, 20, 20, 60]
## Evolving Operators
1. Fitness: sum of importance, because we want to add as much important boxes as possible
3. Constraints: sum of goods weights should be less than or equal to 250.
4. Survive Ratio: 0.5 means only top 50% of population can be survived for next reproducing iteration.
5. Mutation Rate: 0.1 by default, indicating that there is 10% probablity switch from 1 to 0 or inversed.
5. Crossover: crossover strategy can be customized, SinglePointCrossover is usually proper for binary cases. 


In [2]:
def total_size(individual, size=np.array([])) :
    chr_arr = np.array(individual.chromosome)
    siz_arr = np.array(size)
    total = np.dot(chr_arr, siz_arr.T)
    return total 

def total_size_lt250(individual, size=np.array([])) :
    total = total_size(individual, size)
    return total <= 250

# Define Hyperparameters 
- Initial Individuals: randomly initialized individuals will involve more possibility to find the best solution.
- Selection ratio: can be changed to a smaller value to make the algorithm faster and a larger value to keep more candidates. 


# Algorithm Steps
1. Initialize: input parameters and create algorithm instance
1. Calculate fitness: calculate fitness value for every individual
1. Select: keep only the individuals fulfill the constrants
1. Reproduce: generate new individuals by mutation and crossover operator
1. Exit criteria: check if stop criteria is fulfilled. If yes, stop the progress, otherwise repeat from step 2.  

In [3]:
box_importance = [6, 5, 8, 7, 6, 9, 4, 5, 4, 9, 2, 1]
box_weights = [20, 30, 60, 90, 50, 70, 30, 30, 70, 20, 20, 60]


In [4]:
individuals = [ 
    BinaryIndividual([1,1,1,0,0,0,0,0,0,0,0,1],0,0),
    BinaryIndividual([1,0,0,0,1,0,0,0,0,0,0,1],0,0),
    BinaryIndividual([0,0,0,0,0,1,1,0,0,1,0,0],0,0),
    BinaryIndividual([0,0,1,0,0,0,0,0,1,0,0,1],0,0),
    BinaryIndividual([0,1,0,0,1,0,0,0,0,0,0,1],0,0),
]    

In [5]:
sel = LeadingSelector(
    ratio = 0.5,
    constraints=[lambda x: total_size_lt250(x, box_weights)]
)

fit = WeightedSumFitness(weights = box_importance)
xo = SinglePointCrossOver()

In [6]:
env = Environment(
    individuals,
    selector=sel,
    crossover=xo, 
    fitness_func=fit,
    MAX_GENERATION=50,
    CAPACITY=100, 
    MAX_ITERATION=100,
    # log_level='debug'
    verbose=1,
)

env.evolute()

print(env.species.population(), env.species.generations())


2022-11-13 14:55:35,746 - Species - INFO - ITERATION START -- : 0
2022-11-13 14:55:35,749 - Species - DEBUG - FITNESS - top:22; sum: 80; avg:16.0; population:5
2022-11-13 14:55:35,751 - Species - DEBUG - SELECTION -- top:22; sum: 55; avg:18.333333333333332; population:3
2022-11-13 14:55:35,752 - Species - DEBUG - PUNISHMENT -- population: 3, diversity:0
2022-11-13 14:55:35,755 - Species - DEBUG - MUTATION -- population: 6; generation: 0
2022-11-13 14:55:35,759 - Species - DEBUG - XOVER -- population: 36; generation: 1
2022-11-13 14:55:35,765 - Species - DEBUG - FEASIBLE -- top:22; sum: 55; avg:18.333333333333332; population:30
2022-11-13 14:55:35,767 - Species - DEBUG - FITNESS - top:28; sum: 535; avg:17.833333333333332; population:30
2022-11-13 14:55:35,769 - Species - DEBUG - SELECTION -- top:28; sum: 331; avg:22.066666666666666; population:15
2022-11-13 14:55:35,770 - Species - DEBUG - PUNISHMENT -- population: 15, diversity:0
2022-11-13 14:55:35,772 - Species - DEBUG - MUTATION -- 

KeyboardInterrupt: 

In [ ]:
print('The best 3 solutions are: ')
for sol in env.getSolution(3) :
    print(sol) 


The best 3 solutions are: 
[1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
